In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
from datetime import datetime as dt

In [3]:
from decouple import config

In [4]:
from casestudy.update import pull, test, push

In [48]:
pull()

Instantiate new exception logger
Running italy
Running braz
Running US
Running rest
Running austests
Running cadtests
Running ustests
Running resttests
Running strindex
Running gmobi


/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


Aruba AW
Cote d'Ivoire 
Cape Verde CV
Laos LA
North Macedonia MK
Réunion RE
Tajikistan TJ
Running amobi
Instantiating Chrome WebDriver...
Running msmts
Exception('no data is available within your requested subset. Request returned no data.')
Running pollutants


In [49]:
test()

Instantiate new exception logger
making baseframe
initializing
adding fatalities
adding cases
adding tests
adding pollutants
here????????
here?
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE
Region Consistency test
Not Na test for cases
Not Na test for deaths
Not Na test for tests
Exception(' ...Failing Regions: [407]')
Casestudy tests...
Test duplicate dates
Test duplicate days
Test negative days


In [51]:
push()

Merge logs...
Reading merge log file...
No critical errors. Saving baseframe to disk.
initializing
adding fatalities
adding cases
adding tests
adding pollutants
here????????
here?
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
saving...
COMPLETE
Updating readme
push to git
send log email


NameError: name 'logfile' is not defined